In [1]:
import warnings
warnings.simplefilter("ignore")

In [2]:
import sys
import time
import random
import numpy as np
from sklearn import cluster

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=T

In [3]:
def update_DS(datapoint_idx, datapoint_features, cluster_idx):
    DISCARD_SET_stats[cluster_idx]['point_ids'].append(datapoint_idx)
    DISCARD_SET_stats[cluster_idx]['N'] = DISCARD_SET_stats[cluster_idx]['N']+1
    for dim in range(dataset_dimensionality):
        DISCARD_SET_stats[cluster_idx]['sum'][dim] = DISCARD_SET_stats[cluster_idx]['sum'][dim]+datapoint_features[dim]
        DISCARD_SET_stats[cluster_idx]['sumsq'][dim] = DISCARD_SET_stats[cluster_idx]['sumsq'][dim]+datapoint_features[dim]**2
    DISCARD_SET_stats[cluster_idx]['stdev'] = np.sqrt((DISCARD_SET_stats[cluster_idx]['sumsq'][:]/DISCARD_SET_stats[cluster_idx]['N']) - (np.square(DISCARD_SET_stats[cluster_idx]['sum'][:])/(DISCARD_SET_stats[cluster_idx]['N']**2)))
    DISCARD_SET_stats[cluster_idx]['centroid'] = DISCARD_SET_stats[cluster_idx]['sum']/DISCARD_SET_stats[cluster_idx]['N']

In [4]:
def update_CS(datapoint_idx, datapoint_features, cluster_idx):
    COMPRESSION_SET_stats[cluster_idx]['point_ids'].append(datapoint_idx)
    COMPRESSION_SET_stats[cluster_idx]['N'] = COMPRESSION_SET_stats[cluster_idx]['N']+1
    for dim in range(dataset_dimensionality):
        COMPRESSION_SET_stats[cluster_idx]['sum'][dim] = COMPRESSION_SET_stats[cluster_idx]['sum'][dim]+datapoint_features[dim]
        COMPRESSION_SET_stats[cluster_idx]['sumsq'][dim] = COMPRESSION_SET_stats[cluster_idx]['sumsq'][dim]+datapoint_features[dim]**2
    COMPRESSION_SET_stats[cluster_idx]['stdev'] = np.sqrt((COMPRESSION_SET_stats[cluster_idx]['sumsq'][:]/COMPRESSION_SET_stats[cluster_idx]['N']) - (np.square(COMPRESSION_SET_stats[cluster_idx]['sum'][:])/(COMPRESSION_SET_stats[cluster_idx]['N']**2)))
    COMPRESSION_SET_stats[cluster_idx]['centroid'] = COMPRESSION_SET_stats[cluster_idx]['sum']/COMPRESSION_SET_stats[cluster_idx]['N']

In [5]:
def merge_CS_CS(key1, key2):
    COMPRESSION_SET_stats[key1]['point_ids'].extend(COMPRESSION_SET_stats[key2]['point_ids'])
    COMPRESSION_SET_stats[key1]['N'] = COMPRESSION_SET_stats[key1]['N']+COMPRESSION_SET_stats[key2]['N']
    for dim in range(dataset_dimensionality):
        COMPRESSION_SET_stats[key1]['sum'][dim] = COMPRESSION_SET_stats[key1]['sum'][dim]+COMPRESSION_SET_stats[key2]['sum'][dim]
        COMPRESSION_SET_stats[key1]['sumsq'][dim] = COMPRESSION_SET_stats[key1]['sumsq'][dim]+COMPRESSION_SET_stats[key2]['sumsq'][dim]
    COMPRESSION_SET_stats[key1]['stdev'] = np.sqrt((COMPRESSION_SET_stats[key1]['sumsq'][:]/COMPRESSION_SET_stats[key1]['N']) - (np.square(COMPRESSION_SET_stats[key1]['sum'][:])/(COMPRESSION_SET_stats[key1]['N']**2)))
    COMPRESSION_SET_stats[key1]['centroid'] = COMPRESSION_SET_stats[key1]['sum']/COMPRESSION_SET_stats[key1]['N']

In [6]:
def merge_CS_DS(keyCS, keyDS):
    DISCARD_SET_stats[keyDS]['point_ids'].extend(COMPRESSION_SET_stats[keyCS]['point_ids'])
    DISCARD_SET_stats[keyDS]['N']= DISCARD_SET_stats[keyDS]['N']+COMPRESSION_SET_stats[keyCS]['N']
    for dim in range(dataset_dimensionality):
        DISCARD_SET_stats[keyDS]['sum'][dim]= DISCARD_SET_stats[keyDS]['sum'][dim]+COMPRESSION_SET_stats[keyCS]['sum'][dim]
        DISCARD_SET_stats[keyDS]['sumsq'][dim]= DISCARD_SET_stats[keyDS]['sumsq'][dim]+COMPRESSION_SET_stats[keyCS]['sumsq'][dim]
    DISCARD_SET_stats[keyDS]['stdev']= np.sqrt((DISCARD_SET_stats[keyDS]['sumsq'][:]/DISCARD_SET_stats[keyDS]['N']) - (np.square(DISCARD_SET_stats[keyDS]['sum'][:])/(DISCARD_SET_stats[keyDS]['N']**2)))
    DISCARD_SET_stats[keyDS]['centroid']= DISCARD_SET_stats[keyDS]['sum']/DISCARD_SET_stats[keyDS]['N']

In [7]:
def point_cluster_mahalanobis_distance(point_features, cluster_stats):
    
    stdev = cluster_stats['stdev'].astype(np.float64)
    centroid = cluster_stats['centroid'].astype(np.float64)
    mahalanobis_distance = 0
    for dim in range(dataset_dimensionality):
        mahalanobis_distance += ((point_features[dim] - centroid[dim]) / stdev[dim])**2
    mahalanobis_distance = np.sqrt(mahalanobis_distance)

    return mahalanobis_distance

In [8]:
def intercluster_mahalanobis_distance(left_cluster_stats, right_cluster_stats): # takes the dict value
    
    stdev1, centroid1 = left_cluster_stats['stdev'], left_cluster_stats['centroid']
    stdev2, centroid2 = right_cluster_stats['stdev'], right_cluster_stats['centroid']

    # Calculate the mahalanobis distance between the clusters
    left_cluster_dist = 0
    right_cluster_dist = 0
    for dim in range(dataset_dimensionality):
        left_cluster_dist += ((centroid1[dim] - centroid2[dim]) / stdev2[dim])**2
        right_cluster_dist += ((centroid2[dim] - centroid1[dim]) / stdev1[dim])**2
    left_cluster_dist = np.sqrt(left_cluster_dist)
    right_cluster_dist = np.sqrt(right_cluster_dist)

    return min(left_cluster_dist, right_cluster_dist)

In [9]:
input_file = "publicdata/hw6_clustering.txt"
n_cluster = 10
output_file = 'output.txt'

In [48]:
# Read the input file (without using spark!)
with open(input_file, 'r') as f_in:
    dataset = np.array(f_in.readlines())
    
# Downsample the dataset for development
#dataset = np.random.choice(dataset, replace=False, size=int(len(dataset)*0.1))

In [11]:
# Initialize the objects for the different sets
RETAINED_SET = dict()
COMPRESSION_SET = dict()
DISCARD_SET = list()
CLUSTERS = dict()

In [12]:
# Dicts to store variable mappings
point_to_encoding, encoding_to_point, cluster_to_encoding = {}, {}, {}

In [13]:
# Randomly sample 20% of the points
round_1_samples = np.random.choice(dataset, replace=False, size=int(len(dataset)*0.2))
round_1_samples

In [14]:
# Store all samples for the initialization step in a list
initialization_step_samples = list()

In [15]:
# Then iterate through the round_1_samples and populate the objects created
encoding_DS = 0
for sample in round_1_samples:
    sample = sample.replace("\n","").split(",")
    sample_id = sample[0]
    features = sample[2:]
    point_to_encoding[str(features)] = sample_id
    encoding_to_point[sample_id] = features
    cluster_to_encoding[encoding_DS] = sample_id
    initialization_step_samples.append(features)
    encoding_DS += 1

In [16]:
# Get the dataset's dimensionality 
dataset_dimensionality = np.array(initialization_step_samples).shape[1]

In [17]:
# Calculate the Mahalanobis threshold to be useda
mahalanobis_threshold = 2 * np.sqrt(dataset_dimensionality)

In [97]:
# Create the K-Means training data from the round_1_samples
X_train = np.array(initialization_step_samples)
X_train

array([['-44.93317998940097', '32.340767954085834', '47.61095720444176',
        ..., '-22.782063025306933', '-42.76851514856648',
        '-46.44279737352166'],
       ['-127.85478182803433', '-124.98538767214734',
        '-115.50252707420971', ..., '124.56768513909584',
        '-122.92451681880523', '106.74640667095056'],
       ['81.65980788749954', '-85.3901152862115', '87.24747569543042',
        ..., '-79.9397471707509', '-77.1847595843751',
        '89.88348174740597'],
       ...,
       ['-37.29333962540038', '39.66464045401867', '52.27855776753009',
        ..., '-36.07914289468184', '-48.72961950178917',
        '-39.67836513357083'],
       ['125.74733471647818', '-120.54899992165402',
        '123.12999635624799', ..., '119.66560193785145',
        '-128.41641426307172', '-119.49748413440165'],
       ['-130.1073776798164', '-115.51867667964405',
        '-119.06694012466286', ..., '120.61306933670173',
        '-130.34769998914217', '120.87717760522321']], dtype='<U21')

In [19]:
# Run K-Means (e.g., from sklearn) with a large K (e.g., 5 times of the number of the input clusters)
kmeans = cluster.KMeans(n_clusters=5*n_cluster)
kmeans_clusters_1 = kmeans.fit_predict(X_train)
kmeans_cluster_centers = kmeans.cluster_centers_

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

...

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

In [98]:
# Assign points to clusters
clusters = dict()

for idx, c_id in enumerate(kmeans_clusters_1):
    # Take the samples for the initialization_step_samples
    datapoint = initialization_step_samples[idx]
    # If there is no key for the current cluster id, create one with the first sample
    if clusters.get(c_id) == None:
        clusters[c_id] = [datapoint]
    # If they key already exists, create more samples
    else:
        clusters[c_id].append(datapoint)

clusters        

IndexError: list index out of range

In [21]:
# Move lone points to the Retention Set
for c_id, datapoints in clusters.items():
    if(len(datapoints) == 1):
        idx = initialization_step_samples.index(datapoints[0])
        RETAINED_SET[cluster_to_encoding[idx]] = datapoints[0]
        initialization_step_samples.remove(datapoints[0])
        # Fix the cluster to encoding dict after a cluster has been exhaust (because its only point was removed)
        for k in range(idx, len(cluster_to_encoding)-1):
            cluster_to_encoding[k] = cluster_to_encoding[k+1]

In [22]:
# Cluster the round_1_samples
X_train = np.array(initialization_step_samples)
kmeans = cluster.KMeans(n_clusters=n_cluster)
kmeans_clusters_2 = kmeans.fit_predict(X_train)
cluster_centers= kmeans.cluster_centers_

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

In [23]:
for idx, c_id in enumerate(kmeans_clusters_2):
    datapoint = initialization_step_samples[idx]
    # If there is no key for the current cluster id, create one with the first sample
    if CLUSTERS.get(c_id) == None:
        CLUSTERS[c_id] = [idx]
    # If they key already exists, create more samples
    else:
        CLUSTERS[c_id].append(idx)

In [109]:
X_train

array([['-44.93317998940097', '32.340767954085834', '47.61095720444176',
        ..., '-22.782063025306933', '-42.76851514856648',
        '-46.44279737352166'],
       ['-127.85478182803433', '-124.98538767214734',
        '-115.50252707420971', ..., '124.56768513909584',
        '-122.92451681880523', '106.74640667095056'],
       ['81.65980788749954', '-85.3901152862115', '87.24747569543042',
        ..., '-79.9397471707509', '-77.1847595843751',
        '89.88348174740597'],
       ...,
       ['-37.29333962540038', '39.66464045401867', '52.27855776753009',
        ..., '-36.07914289468184', '-48.72961950178917',
        '-39.67836513357083'],
       ['125.74733471647818', '-120.54899992165402',
        '123.12999635624799', ..., '119.66560193785145',
        '-128.41641426307172', '-119.49748413440165'],
       ['-130.1073776798164', '-115.51867667964405',
        '-119.06694012466286', ..., '120.61306933670173',
        '-130.34769998914217', '120.87717760522321']], dtype='<U21')

In [112]:
DISCARD_SET_stats[0]

{'points': ['76708',
  '208136',
  '4160',
  '234059',
  '71875',
  '40622',
  '58556',
  '64600',
  '170393',
  '311450',
  '14791',
  '170152',
  '224479',
  '186985',
  '219178',
  '225585',
  '179886',
  '198230',
  '210898',
  '300145',
  '69235',
  '161106',
  '139900',
  '180598',
  '307772',
  '84315',
  '39425',
  '13427',
  '123821',
  '122297',
  '6451',
  '286537',
  '58370',
  '84765',
  '211390',
  '15869',
  '38119',
  '51497',
  '311405',
  '151692',
  '183348',
  '156943',
  '182326',
  '97008',
  '37135',
  '201714',
  '193986',
  '147529',
  '162781',
  '150109',
  '264270',
  '229051',
  '69624',
  '257714',
  '20671',
  '166030',
  '28072',
  '309495',
  '44070',
  '149533',
  '124342',
  '301979',
  '150898',
  '124593',
  '14785',
  '78155',
  '94317',
  '162858',
  '136250',
  '189258',
  '87604',
  '197709',
  '307939',
  '154467',
  '296940',
  '130406',
  '321473',
  '102895',
  '270403',
  '268787',
  '145330',
  '218109',
  '148357',
  '4662',
  '275265',
 

In [24]:
# Populate the DISCARD_SET from the clusters generated from the initialization round
DISCARD_SET_stats = dict()
for key, value in CLUSTERS.items():
    DISCARD_SET_stats[key] = {}
    DISCARD_SET_stats[key]['point_ids'] = []
    for datapoint in value:
        DISCARD_SET_stats[key]['point_ids'].append(cluster_to_encoding[datapoint])
    DISCARD_SET_stats[key]['N'] = len(DISCARD_SET_stats[key]['point_ids'])
    DISCARD_SET_stats[key]['sum'] = np.sum(X_train[value,:].astype(np.float), axis=0)
    DISCARD_SET_stats[key]['sumsq'] = np.sum((X_train[value,:].astype(np.float))**2, axis = 0) 
    DISCARD_SET_stats[key]['stdev'] = np.sqrt((DISCARD_SET_stats[key]['sumsq'][:]/DISCARD_SET_stats[key]['N']) - (np.square(DISCARD_SET_stats[key]['sum'][:])/(DISCARD_SET_stats[key]['N']**2)))
    DISCARD_SET_stats[key]['centroid']= DISCARD_SET_stats[key]['sum']/DISCARD_SET_stats[key]['N']
       


In [25]:
RS_points = list()
for key, value in RETAINED_SET.items():
    RS_points.append(value)

In [115]:
RETAINED_SET

{'52454': ['511.811174895178',
  '484.51001851329863',
  '725.7352627508951',
  '474.04105776360893',
  '760.5702044206341',
  '484.35502650116024',
  '580.6851085561406',
  '743.0675772536231',
  '716.070229415848',
  '902.6725941756604']}

In [26]:
X_train = np.array(RS_points)
if X_train.shape[0] > 0:
    kmeans = cluster.KMeans(n_clusters=min(5*n_cluster,int(len(RS_points)/2+1))) #(n_clusters=int(len(RS_points)/2+1))
    clusters3= kmeans.fit_predict(X_train)
    rs_clusters = {}
    for idx, c_id in enumerate(clusters3):
        if c_id in rs_clusters:
            rs_clusters[c_id].append(idx)
        else:
            rs_clusters[c_id]= [idx]
# If there are no samples in the RS_points, then there is nothing to cluster
else:
    rs_clusters = {}

C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
C:\Users\Matheus\Anaconda3\lib\site-packages\sklearn\metrics

In [27]:
rs_clusters

{0: [0], 1: [1]}

In [28]:
# Populate the COMPRESSION_SET from the clusters generated from the initialization round
COMPRESSION_SET_stats= {}
for key, value in rs_clusters.items():
    # Only clusters with more than one sample in them go to the CS, those with only 1 sample are RS 
    if len(value) > 1:
        COMPRESSION_SET_stats[key]= {}
        
        COMPRESSION_SET_stats[key]['point_ids']= []
        for idx in value:
            datapoint_idx= list(RETAINED_SET.keys())[list(RETAINED_SET.values()).index(RS_points[idx])]
            COMPRESSION_SET_stats[key]['point_ids'].append(datapoint_idx)
        COMPRESSION_SET_stats[key]['N']= len(value) 
        COMPRESSION_SET_stats[key]['sum']= np.sum(X_train[value,:].astype(np.float), axis = 0) 
        COMPRESSION_SET_stats[key]['sumsq']= np.sum((X_train[value,:].astype(np.float))**2, axis = 0) 
        COMPRESSION_SET_stats[key]['stdev']= np.sqrt((COMPRESSION_SET_stats[key]['sumsq'][:]/COMPRESSION_SET_stats[key]['N']) - (np.square(COMPRESSION_SET_stats[key]['sum'][:])/(COMPRESSION_SET_stats[key]['N']**2)))
        COMPRESSION_SET_stats[key]['centroid']= COMPRESSION_SET_stats[key]['sum']/COMPRESSION_SET_stats[key]['N']

In [29]:
# Clean the samples from the retained set which have been summarized
for key, value in rs_clusters.items():
    if len(value) > 1:
        for datapoint in value:
            dict_key_to_remove= list(RETAINED_SET.keys())[list(RETAINED_SET.values()).index(RS_points[datapoint])]
            del RETAINED_SET[dict_key_to_remove]            

In [30]:
RS_points = list()
for key, value in RETAINED_SET.items():
    RS_points.append(value)

In [31]:
# Objects to tally values to be outputted
n_points_DS = 0 
n_clusters_CS = 0
n_points_CS = 0
n_points_RS = 0

In [32]:
for key, value in DISCARD_SET_stats.items():
	n_points_DS+= len(value['point_ids'])

for key, value in COMPRESSION_SET_stats.items():
	n_clusters_CS+= 1
	n_points_CS+= len(value['point_ids'])

n_points_RS= len(RS_points)

In [33]:
f = open(output_file, "w")
print("Round 1: "+str(n_points_DS)+","+str(n_clusters_CS)+","+str(n_points_CS)+","+str(n_points_RS))
f.write("Round 1: "+str(n_points_DS)+","+str(n_clusters_CS)+","+str(n_points_CS)+","+str(n_points_RS))

Round 1: 6444,0,0,2


19

In [34]:
import warnings
warnings.simplefilter("ignore")

start = time.time()

# Initialization is finished, run the regular iterations
for curr_round in range(2,6):
    
    # Load the samples for the iteration about the begin
    if curr_round < 5:
        iteration_samples = np.random.choice(dataset, replace=False, size=int(len(dataset)*0.2))
    # The last iteration might have a slightly different number of samples
        iteration_samples = np.random.choice(dataset, replace=False, size=len(dataset) - 4*(int(len(dataset)*0.2)))
    
    
    ##################################
    ### Assign Samples to DS/CS/RS ###
    ##################################
    start = time.time()
    
    # Get the starting index for the points in this iterations
    start_index = encoding_DS
    
    # Store the features for each sample
    iteration_samples_features = []
    
    # Update the objects that reference the datapoints
    for sample in iteration_samples:
        sample = sample.replace("\n","").split(",")
        sample_id = sample[0]
        features = sample[2:]
        point_to_encoding[str(features)] = sample_id
        encoding_to_point[sample_id] = features
        cluster_to_encoding[encoding_DS] = sample_id
        iteration_samples_features.append(features)
        encoding_DS += 1
        
    # Attempt to assign points to the DISCARD_SET
    datapoints = np.array(iteration_samples_features).astype(np.float64)
    
    # Iterate over each sample in the datapoints
    for idx, sample in enumerate(datapoints):
                  
        # Get the point's unique ID
        sample_id = cluster_to_encoding[start_index+idx]
        
        # Default to assuming points are in the RETAINED_SET
        assigned_cluster = -1
        
        # Track the lowest distance between the sample and all clusters
        lowest_dist = mahalanobis_threshold
              
        # Find the DISCARD_SET cluster closest to the current sample      
        for cluster_id, cluster_stats in DISCARD_SET_stats.items():
            
            mahalanobis_distance = point_cluster_mahalanobis_distance(sample, cluster_stats)
            
            # If the distance is under the mahalanobis_threshold and also the lowest distance yet found, update the point's cluster
            if mahalanobis_distance < lowest_dist:
                assigned_cluster = cluster_id
                lowest_dist = mahalanobis_distance

        # Update the statistics of the cluster to which the point is assigned 
        if assigned_cluster != -1:
            update_DS(sample_id, sample, assigned_cluster)
            
        # If the sample could not be assigned to any cluster in the DS, try assigning it to the CS
        else:
            
            # Default to assuming points are in the RETAINED_SET
            assigned_cluster = -1
            
            # Track the lowest distance between the sample and all clusters
            lowest_dist = mahalanobis_threshold
            
            # Find the COMPRESSION_SET cluster closest to the current sample
            for cluster_id, cluster_stats in COMPRESSION_SET.items():
                
                mahalanobis_distance = point_cluster_mahalanobis_distance(sample, cluster_stats)
                
                # If the distance is under the mahalanobis_threshold and also the lowest distance yet found, update the point's cluster
                if mahalanobis_distance < lowest_dist:
                    assigned_cluster = cluster_id
                    lowest_dist = mahalanobis_distance

            # Update the statistics of the cluster to which the point is assigned 
            if assigned_cluster != -1:
                update_CS(sample_id, sample, assigned_cluster)

            # If the BRF also failed to assing the sample to a CS cluster, send it to the RS
            else:
                RETAINED_SET[sample_id] = list(sample.astype(str))
                RS_points.append(list(sample.astype(str)))

    print(f'Round {curr_round}. Sample Assignment. Duration: {time.time() - start:.2f} seconds.')            
    ######################################
    ### Create New CSs from RS Samples ###
    ######################################
    start = time.time()
    
    # Training data for KMeans clustering
    X_train = np.array(RS_points)
    kmeans = cluster.KMeans(n_clusters=min(5*n_cluster,int(len(RS_points)/2+1))) #(n_clusters=int(len(RS_points)/2+1))
    clusters4= kmeans.fit_predict(X_train)
    
    # Assign the RS samples to their predicted clusters
    RS_clusters = dict()
    for idx, c_id in enumerate(clusters4):
        # If there is no key for the current cluster id, create one with the first sample
        if RS_clusters.get(c_id) == None:
            RS_clusters[c_id] = [idx]
        # If they key already exists, create more samples
        else:
            RS_clusters[c_id].append(idx)
     
    # Update the COMPRESSION_SET statistics with the newly created CS clusters
    for key, value in RS_clusters.items():
        #print(f'RS Key: {key}. RS value length: {len(value)}')
        
        # If two of more RS samples got clustered together, turn them into a CS
        if len(value) > 1:
            #print(f'Adding RS to CS.')
            # Find the index of the next key to be added to the CS
            k = 0
            if key in COMPRESSION_SET_stats:
                while k in COMPRESSION_SET_stats:
                    k = k + 1
            else:
                k = key
                
            # Create a new entry in the COMPRESSION_SET_stats dict for the new CS
            COMPRESSION_SET_stats[k] = dict()
            
            # Add the merged RS points to the CS
            COMPRESSION_SET_stats[k]['point_ids']= []
            for i in RS_clusters[key]:
                datapoint_idx= list(RETAINED_SET.keys())[list(RETAINED_SET.values()).index(RS_points[i])]
                COMPRESSION_SET_stats[k]['point_ids'].append(datapoint_idx)
            COMPRESSION_SET_stats[k]['N']= len(RS_clusters[key]) 
            COMPRESSION_SET_stats[k]['sum']= np.sum(X_train[RS_clusters[key],:].astype(np.float), axis = 0) 
            COMPRESSION_SET_stats[k]['sumsq']= np.sum((X_train[RS_clusters[key],:].astype(np.float))**2, axis = 0) 
            COMPRESSION_SET_stats[k]['stdev']= np.sqrt((COMPRESSION_SET_stats[k]['sumsq'][:]/COMPRESSION_SET_stats[k]['N']) - (np.square(COMPRESSION_SET_stats[k]['sum'][:])/(COMPRESSION_SET_stats[k]['N']**2)))
            COMPRESSION_SET_stats[k]['centroid']= COMPRESSION_SET_stats[k]['sum']/COMPRESSION_SET_stats[k]['N']
            
    # Erase from the RS_clusters any cluster that was moved to the COMPRESSION_SET
    for key, value in RS_clusters.items():
        if len(value) > 1:
            for item in value:
                dict_key_to_remove= point_to_encoding[str(RS_points[item])]
                if dict_key_to_remove in RETAINED_SET:
                    del RETAINED_SET[dict_key_to_remove]
                    
    # Then take any points which were not assigned to either the DS or CS and store them for the next iteration
    RS_points= [] 
    for key in RETAINED_SET.keys():
        RS_points.append(RETAINED_SET[key])
        
    print(f'Round {curr_round}. Create New CSs. Duration: {time.time() - start:.2f} seconds.')    
    #################################################
    ### Merge CSs Below the Mahalanobis Threshold ###
    #################################################
    start = time.time()
    
    close_CSs = dict()
    for key1, value1 in COMPRESSION_SET_stats.items():
        
        # Default to assuming there is no other CS cluster close by
        assigned_cluster = key1

        # Track the lowest distance between the sample and all clusters
        lowest_dist = mahalanobis_threshold
        
        # Compare each CS cluster to all other CS clusters
        for key2, value2 in COMPRESSION_SET_stats.items():
            
            # Do not compare the a cluster to itself
            if key1 == key2:
                continue
                
            intercluster_dist = intercluster_mahalanobis_distance(value1, value2)
            
            # If the intercluster distance is below the threshold, make the pair a candidate for merging
            if intercluster_dist < lowest_dist:
                assigned_cluster = key2
                lowest_dist = intercluster_dist
                
        # Once all pairwise comparisons were done for a given cluster (key1) store the results
        close_CSs[key1] = assigned_cluster
        
    # Once all closest CS clusters were found, merge them and update the COMPRESSION_SET_stats
    for CS_cluster1, CS_cluster2 in close_CSs.items():
        if CS_cluster1 in COMPRESSION_SET_stats and CS_cluster2 in COMPRESSION_SET_stats and CS_cluster1 != CS_cluster2:
            merge_CS_CS(CS_cluster1, CS_cluster2)
            del COMPRESSION_SET_stats[CS_cluster2]
            
    print(f'Round {curr_round}. Merge CSs Below the Mahalanobis Threshold. Duration: {time.time() - start:.2f} seconds.')        
    ######################################################
    ### At the Final Iteration Merge CSs to Nearby DSs ###
    ######################################################
    start = time.time()
    
    # Check if it is the last iteration
    if curr_round == 5:
        
        close_DSs = dict()
        for CS_key, CS_value in COMPRESSION_SET_stats.items():
            
            # Default to assuming there is no DS cluster close by
            assigned_cluster = None

            # Track the lowest distance between the sample and all clusters
            lowest_dist = mahalanobis_threshold

            for DS_key, DS_value in DISCARD_SET_stats.items():
                
                intercluster_dist = intercluster_mahalanobis_distance(CS_value, DS_value)

                # If the intercluster distance is below the threshold, make the pair a candidate for merging
                if intercluster_dist < lowest_dist:
                    assigned_cluster = DS_key
                    lowest_dist = intercluster_dist
                
        # Once all pairwise comparisons were done for a given cluster (CS_key) store the results
        close_DSs[CS_key] = assigned_cluster
        
        # Once all closest CS clusters were found, merge them and update the COMPRESSION_SET_stats
        for CS_cluster, DS_cluster in close_DSs.items():
            if CS_cluster in COMPRESSION_SET_stats and DS_cluster in DISCARD_SET_stats:
                merge_CS_DS(CS_cluster, DS_cluster)
                del COMPRESSION_SET_stats[CS_cluster]
                
    print(f'Round {curr_round}. Merge CSs to Nearby DSs. Duration: {time.time() - start:.2f} seconds.')            
    ###################################
    ### Output Iteration Statistics ###
    ###################################
    start = time.time()

    n_points_DS = 0 
    n_clusters_CS = 0
    n_points_CS = 0
    n_points_RS = 0
    
    for key, value in DISCARD_SET_stats.items():
        n_points_DS += value['N']

    for key, value in COMPRESSION_SET_stats.items():
        n_clusters_CS += 1
        n_points_CS += value['N']

    n_points_RS = len(RS_points)
    
    print(f'Round {curr_round}. Output Iteration Statistics. Duration: {time.time() - start:.2f} seconds.')    
    
    print("Round "+str(curr_round)+": "+str(n_points_DS)+","+str(n_clusters_CS)+","+str(n_points_CS)+","+str(n_points_RS))
    print()
    f.write("\nRound "+str(curr_round)+": "+str(n_points_DS)+","+str(n_clusters_CS)+","+str(n_points_CS)+","+str(n_points_RS))

Round 2. Starting Sample Assignment. Duration: 0.02 seconds.
sample cluster DS: 5
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 1
updating DS
sample cluster 

updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 7
updating D

sample cluster DS: 8
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 0
updating DS
sample clu

updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 8
updating D

updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 0
updating D

...

updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 5
updating D

sample cluster DS: 7
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 9
updating DS
sample clu

updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 9
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 2
updating D

updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 0
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 4
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 7
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 1
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 6
updating DS
sample cluster DS: 2
updating DS
sample cluster DS: 5
updating DS
sample cluster DS: 8
updating DS
sample cluster DS: 3
updating DS
sample cluster DS: 3
updating D

Round 5. Output Iteration Statistics. Duration: 13.76 seconds.
Round 5: 32227,3,6,1



In [35]:
close_DSs

{2: None}

In [36]:
DISCARD_SET_stats

{9: {'points': ['204880',
   '60408',
   '271907',
   '248056',
   '237133',
   '183147',
   '310749',
   '107532',
   '254678',
   '165384',
   '196141',
   '191381',
   '298953',
   '49774',
   '85670',
   '135858',
   '144136',
   '301311',
   '68454',
   '319882',
   '296738',
   '2342',
   '134112',
   '50213',
   '133076',
   '166876',
   '154767',
   '226653',
   '160206',
   '272754',
   '85857',
   '58855',
   '210269',
   '65576',
   '173915',
   '316830',
   '174379',
   '242837',
   '296826',
   '244788',
   '117626',
   '221128',
   '195765',
   '117560',
   '14226',
   '119363',
   '48295',
   '264199',
   '186344',
   '107863',
   '44613',
   '3246',
   '265209',
   '67432',
   '37348',
   '320861',
   '87450',
   '199104',
   '265476',
   '173649',
   '273623',
   '224520',
   '73488',
   '262163',
   '205902',
   '145543',
   '277557',
   '14822',
   '275351',
   '237821',
   '152587',
   '295647',
   '169418',
   '260847',
   '48255',
   '208785',
   '279452',
   '104

In [37]:
# Write clustering results
f.write("\n")
f.write("\nThe clustering results")
clusterlist= {}
for key, value in DISCARD_SET_stats.items():
	for point in value['point_ids']:
		clusterlist[point]= key
for key, value in COMPRESSION_SET_stats.items():
	for point in value['point_ids']:
		clusterlist[point]=-1
for key in RETAINED_SET:
	clusterlist[key]=-1

for key in sorted(clusterlist.keys(), key=int):
	f.write("\n"+str(key)+","+str(clusterlist[key]))

In [38]:
print(len(clusterlist))
f.close()

end= time.time()
print("Duration: "+str(end-start))

19125
Duration: 14.429314374923706


In [39]:
print(len(DISCARD_SET_stats[0]['point_ids']))
print(len(DISCARD_SET_stats[1]['point_ids']))
print(len(DISCARD_SET_stats[2]['point_ids']))
print(len(DISCARD_SET_stats[3]['point_ids']))
print(len(DISCARD_SET_stats[4]['point_ids']))
print(len(DISCARD_SET_stats[5]['point_ids']))
print(len(DISCARD_SET_stats[6]['point_ids']))
print(len(DISCARD_SET_stats[7]['point_ids']))
print(len(DISCARD_SET_stats[8]['point_ids']))
print(len(DISCARD_SET_stats[9]['point_ids']))

3099
3710
3130
3193
2388
3671
2207
3408
4117
3304
